# Introduction to permutations 

In [1]:
%cd ../
import numpy as np
from core.asym_verification_protocol import permutation_utils

/Users/gru/Documents/skripte/CSM/scripts/ws_15_16/38777 Masterthesis/src/audio_cwe_framework


## Construct a permutation by sorting a list of random numbers
A step-by-step illustration

In [2]:
# Form a 2d-array, where the first element is 'random' value and the second one denotes the initial index
a = np.array([[0.8823, 0],[0.453, 1], [0.9823, 2],[0.533, 3]])
print('Original array:\n', a)

# Slice the array, [:,0] removes the indices
# and use argsort(), which returns the indices, that would sort the list
sort_ids = (a[:,0].argsort())

# Sort the original array by using the received indices
b = a[sort_ids]
print('Sorted array:\n', b)

# The sorting process constitutes a permutation
print("----------------------------------------------------------------")

print('The resulting permutation:\n', np.arange(0,len(sort_ids),1))
print('',sort_ids) # having a 1d array of random numbers a.argsort() gives the permutation


Original array:
 [[ 0.8823  0.    ]
 [ 0.453   1.    ]
 [ 0.9823  2.    ]
 [ 0.533   3.    ]]
Sorted array:
 [[ 0.453   1.    ]
 [ 0.533   3.    ]
 [ 0.8823  0.    ]
 [ 0.9823  2.    ]]
----------------------------------------------------------------
The resulting permutation:
 [0 1 2 3]
 [1 3 0 2]


## Generate a permutation by using the logistic map
The code above shows how simple it is, to derive a permutation from an array of random numbers. One way to generate those random numbers is chaos, which has a lot of desirable attributes and enables a compact description of the permutation by its key parameters. One chaotic equation is the so called logistic map. 
$$
x_{k+1} = \mu x_k(1-x_k)
$$


In order get complex chaotic behaviour choose $3.569945\dots<mu < 4$

In [3]:
# %load -s log_map ./core/asym_verification_protocol/permutation_utils.py
def log_map(x, mu):
    """Logistic map equation. Takes x_k and mu and calculates x_{k+1}

    :param x: the current value
    :param mu: 0<\mu<4; to get complex dynamic behaviour choose 3.569945..<mu<4
    :return: x_{k+1}
    """
    return mu * x * (1 - x)


In [4]:
# Generate random numbers with the use of the logistic map
x_i = 0.7  # start value
m = 100  # forerunning iterations
mu = 3.7 

size = 5 
n = m + size
randoms = []
i = 0
while i < n:
    x_i = log_map(x_i, mu)
    if i >= m:
        randoms.append(x_i)
    i += 1

print('Resulting pseudo-random numbers:\n', randoms)

Resulting pseudo-random numbers:
 [0.5682177680599102, 0.9077814436474199, 0.30974288860225674, 0.7910682567815709, 0.6115332986083089]


## Compose two permutations

In [5]:
# %load -s compose_permutations ./core/asym_verification_protocol/permutation_utils.py
def compose_permutations(p2, p1):
    """Calculates p2 \circ p1 and returns the resulting permutation.

    :param p1: a permutation
    :param p2: another permutation
    :return: p3: the resulting permutation p2 \circ p1
    """
    p3 = np.zeros_like(p1)

    for i, v in enumerate(p1):
        p3[i] = p2[v]

    return p3


In [6]:
p1 = np.array([3, 1, 0, 4, 2])
print('p1:\n', p1)
p2 = np.array([4, 0, 2, 1, 3])
print('p2:\n', p2)

p3 = compose_permutations(p2, p1)
print('-------------------------')
print('p2 \circ p1:\n', p3)



p1:
 [3 1 0 4 2]
p2:
 [4 0 2 1 3]
-------------------------
p2 \circ p1:
 [1 0 4 3 2]


## Generate partial permutation 
Bring both concepts together:
Form two arbitrary permutations $\tau$ and $\rho$ and find a partial permutation $\sigma$, so that $\tau = \sigma \circ \rho$ is valid. This is accomplished by calculating $sigma = \tau \circ \rho^{-1}$.

In [7]:
# Generate a permutation \tau by the use of a logistic map with the following params
size = 6
x_i = 0.7
mu = 3.7
i = 100

randoms = []
while i>0:
    x_i = log_map(x_i, mu)
    if i<=size:
        randoms.append(x_i)
    i -= 1

tau = np.array(randoms).argsort()
print("----------------------------------------------------------------")
print("Tau:\n",tau)

# Generate a second permutation \rho by the use of a logistic map with the following params
mu = 3.59
x_i = 0.3
i = 100

randoms = []
while i>0:
    x_i = log_map(x_i, mu)
    if i<=size:
        randoms.append(x_i)
    i -= 1

rho = np.array(randoms).argsort()
print("Rho:\n", rho)

# Now find \sigma

# Invert rho
inv_rho = np.zeros(len(rho), dtype=rho[0].dtype)
for i, v in enumerate(rho):
    inv_rho[v] = i

# Calculate \tau \circ \rho^{-1} 
sigma = np.zeros(size, dtype=np.int)
for i, v in enumerate(inv_rho):
        sigma[i] = tau[v]

print("Sigma:\n", sigma)
print("----------------------------------------------------------------")



----------------------------------------------------------------
Tau:
 [2 0 4 3 5 1]
Rho:
 [3 1 5 4 0 2]
Sigma:
 [5 0 1 2 3 4]
----------------------------------------------------------------


Let's check $\tau = \sigma \circ \rho$

In [8]:
p = compose_permutations(sigma, rho)
print('Result:\n', p)
if np.array_equal(p, tau):
    print('\\tau == \\sigma \\circ \\rho')

Result:
 [2 0 4 3 5 1]
\tau == \sigma \circ \rho


Now apply the permutations to a list of nodes.

In [9]:
# Generate a vector of nodes to permute
nodes = np.zeros(size, dtype=np.uint32)
for i in range(len(nodes)):
    nodes[i] = i

print("Nodes:\n", nodes)

# Apply permutation rho
shuffled_nodes = np.zeros(size, dtype=np.int)

for i, v in enumerate(rho):
    shuffled_nodes[v] = nodes[i]

print("\\rho(nodes):\n", shuffled_nodes)

# Apply permutation sigma
shuffled_nodes_b = np.zeros(size, dtype=np.int)

for i,v in enumerate(sigma):
    shuffled_nodes_b[v] = shuffled_nodes[i]

print("\\sigma\\rho(nodes)):\n", shuffled_nodes_b)

print("----------------------------------------------------------------")

# Apply permutation tau
shuffled_nodes_c = np.zeros(size, dtype=np.int)

for i,v in enumerate(tau):
    shuffled_nodes_c[v] = nodes[i]

print("Nodes:\n", nodes)
print("\\tau(nodes):\n", shuffled_nodes_c)

print("----------------------------------------------------------------")
if np.array_equal(shuffled_nodes_b, shuffled_nodes_c):
    print('\\tau(n) == \\sigma(\\rho(n))')



Nodes:
 [0 1 2 3 4 5]
\rho(nodes):
 [4 1 5 0 3 2]
\sigma\rho(nodes)):
 [1 5 0 3 2 4]
----------------------------------------------------------------
Nodes:
 [0 1 2 3 4 5]
\tau(nodes):
 [1 5 0 3 2 4]
----------------------------------------------------------------
\tau(n) == \sigma(\rho(n))
